# Data Pre-processing
- Download MJM_datakind.csv, DON_datakind.csv, SDN_datakind.csv to 'data' folder
- Download click data 'motherjones_clicks_2017_datakind' and 'motherjones_clicks_2018_datakind' to 'data' folder
- Run this script to clean and reformat raw data files before running grouped insights scripts
- Take less than 5min


## load packages

In [ ]:
import pandas as pd
import datetime
import os
from time import strptime
import re

#  get the path for 'data' directory that's on the same level with the script directory
# 'data' directory should already contain all the raw data files
#  all the processed data will be saved in 'data' directory.
data_folder = os.path.abspath(os.path.join(os.getcwd(),'../data'))
print('Your data folder is:' + data_folder)

## 1. Filter user files and save the data to new files
- remove the last month of data to avoid bias introduced by incomplete last month data
- remove donation records with paid amount euqal to zero
- save the filtered as a new file named as orignal name + '_filtered.csv'
- e.g. 'MJM_datakind.csv' will be filtered and saved as 'MJM_datakind_filtered.csv'

In [ ]:
# the absolute paths for donation files and subscription file
raw_filenames = [os.path.join(data_folder, f) for f in ['MJM_datakind.csv', 'DON_datakind.csv', 'SDN_datakind.csv']]

# the column name for order date
date_column = 'ORD ENTR DT'
# the column name for the paid dollar amount
amt_column = 'AMT PAID'

# process the files one by one 
for f in raw_filenames:
    
    print("processing " + f)
    if '.csv' in f:
        try:
            dat = pd.read_csv(f,encoding = "utf-8")
        except UnicodeDecodeError:
            dat = pd.read_csv(f,encoding = "ISO-8859-1")    

    if '.xlsx' in f:
        dat = pd.read_excel(f)
    
    # convert the date column to datetime format
    dat[date_column] = pd.to_datetime(dat[date_column])
        
    # get the latest month from the file
    last_date = datetime.datetime((max(dat[date_column])).year,
                                  (max(dat[date_column])).month,                                     
                                   1)
        
    # remove the last month of data to avoid bias introduced by incomplete last month data
    print("remove data later than " + str(last_date) + ": removed " + str(sum(dat[date_column] >last_date)) + " records")
    dat_filtered = dat[dat[date_column] < last_date]
    
    # remove donation records with paid amount euqal to zero
    if 'DON' in f or 'SDN' in f:
        print("remove data with AMT PAID = 0"  + ": removed " + str(sum(dat_filtered[amt_column] <= 0)) + " records")
        dat_filtered = dat_filtered[dat_filtered[amt_column] > 0]

    # save the data in a new file with orignal name + '_filtered.csv'
    print("create " + f.replace('.csv','_filtered.csv'))
    dat_filtered.to_csv(f.replace('.csv','_filtered.csv'))
    print('\n')

## 2. Filter click data files, combine and save the data to a new file

In [ ]:
%%time
# the raw 2017 click data is original saved in folder 'motherjones_clicks_2017_datakind'
path_2017 = os.path.join(data_folder,'motherjones_clicks_2017_datakind')
# the raw 2018 click data is original saved in folder 'motherjones_clicks_2018_datakind'
path_2018 = os.path.join(data_folder,'motherjones_clicks_2018_datakind')

# prefix for 2017 data files
file_prefix_2017 = 'datakind_2017_'
# prefix for 2018 data files
file_prefix_2018 = 'datakind_2018_'

# months extracted from 2017 data filenames
filenames_2017 = ['january','february','april','april 2','may','june','july1','july2','august','septempber','october','november','december']
# months extracted from 2018 data filenames
filenames_2018 = ['january','feb','march','april','may','june','july','august','september','october','november']


df = pd.DataFrame()
df_list = []
for f in filenames_2017:
    
    current_file = os.path.join(path_2017,file_prefix_2017 + f + '.csv')
    tmp = pd.read_csv(current_file, index_col=None, header=0)
    
    # correct month name typos
    if f == 'septempber':
        f = 'september'
    
    if f == 'feb':
        f = 'february'
        
    tmp['MONTH'] = f
    tmp['YEAR'] = 2017
    
    # convert month name to numeric format and combine month and year and save as datetime
    tmp['file_month'] = datetime.datetime(2017, int(strptime(re.sub('[0-9]+','',f.replace(' ','')),'%B').tm_mon),1)
    df_list.append(tmp) 
    print("processed: " + current_file)
    
for f in filenames_2018:
    
    current_file = os.path.join(path_2018,file_prefix_2018 + f + '.csv')
    tmp = pd.read_csv(current_file, index_col=None, header=0)
    
    # correct month name typos
    if f == 'septempber':
        f = 'september'
    
    if f == 'feb':
        f = 'february'
        
    tmp['MONTH'] = f
    tmp['YEAR'] = 2018
    
    # convert month name to numeric format and combine month and year and save as datetime
    tmp['file_month'] = datetime.datetime(2018, int(strptime(re.sub('[0-9]+','',f.replace(' ','')),'%B').tm_mon),1)
    df_list.append(tmp)
    print("processed: " + current_file)

print("combining processed data...")
df = pd.concat(df_list)
## preprocessing
# remove irrelevant columns, rename columns
df = df.drop('email', axis=1)
df = df.drop('email hash', axis=1)
df.rename(columns = {'email hashed': 'EMAIL',
                     'url': 'URL'
                     }, inplace=True)
# only keep URLs containing www.motherjones.com
email_clean = df[df['URL'].str.contains("www.motherjones.com")]

print("saving processed data...")
email_clean.to_csv(os.path.join(data_folder,'email_clean_2017_18.csv'))
print("cleaned and combined click data; save as " + os.path.join(data_folder,'email_clean_2017_18.csv'))